In [1]:
import numpy as np
from math import sqrt

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [2]:
# 4 tokens Let's to go <eos>
# 4 chinese tokens 我 出发 去 <eos>

input_lets = np.array([1.87, 1.09])
input_go = np.array([-1.68, 0.67])

output_chufa = np.array([-0.86, 2.37])
output_eos = np.array([2.7, -0.34])


W_q_encoder = np.array([[ 1.1, -2.8], 
                        [ 0.6,  2.4]])
W_k_encoder = np.array([[-1.7, -1.4],
                        [ 0.5,  0.9]])
W_v_encoder = np.array([[ 1.5, -0.3],
                        [-1.0, -0.2]])

# Train encoder 
q_1_encoder = np.dot(W_q_encoder, input_lets)
q_2_encoder = np.dot(W_q_encoder, input_go)
k_1_encoder = np.dot(W_k_encoder, input_lets)
k_2_encoder = np.dot(W_k_encoder, input_go)
v_1_encoder = np.dot(W_v_encoder, input_lets)
v_2_encoder = np.dot(W_v_encoder, input_go)

In [3]:
print(q_1_encoder)
print(q_2_encoder)
print(k_1_encoder)
print(k_2_encoder)
print(v_1_encoder)
print(v_2_encoder)

[-0.995  3.738]
[-3.724  0.6  ]
[-4.705  1.916]
[ 1.918 -0.237]
[ 2.478 -2.088]
[-2.721  1.546]


In [4]:
qk_encoder_1 = np.array([np.dot(q_1_encoder, k_1_encoder.T), np.dot(q_1_encoder, k_2_encoder.T)])
print(qk_encoder_1)

[11.843483 -2.794316]


In [5]:
softmax_qk_encoder_1 = softmax(qk_encoder_1)
print(softmax_qk_encoder_1)
output_encoder_1 = np.dot(softmax_qk_encoder_1, np.array([v_1_encoder, v_2_encoder]))
print(output_encoder_1)

[9.99999561e-01 4.39424711e-07]
[ 2.47799772 -2.0879984 ]


In [6]:
qk_encoder_2 = np.array([np.dot(q_2_encoder, k_1_encoder.T), np.dot(q_2_encoder, k_2_encoder.T)])
print(qk_encoder_2)

[18.67102  -7.284832]


In [7]:
softmax_qk_encoder_2 = softmax(qk_encoder_2)
print(softmax_qk_encoder_2)
output_encoder_2 = np.dot(softmax_qk_encoder_2, np.array([v_1_encoder, v_2_encoder]))
print(output_encoder_2)

[1.0000000e+00 5.3396981e-12]
[ 2.478 -2.088]


In [8]:
resi_encoder_1 = output_encoder_1 + input_lets
resi_encoder_2 = output_encoder_2 + input_go
print(resi_encoder_1)
print(resi_encoder_2)

[ 4.34799772 -0.9979984 ]
[ 0.798 -1.418]


In [9]:
W_q_decoder = np.array([[ 0.4, -0.3],
                        [ 0.4,  0.1]])
W_k_decoder = np.array([[ 0.4, -0.4],
                        [-0.7,  0.3]])
W_v_decoder = np.array([[-1.1, -0.4],
                        [-0.7,  1.3]])

# Train decoder
q_eos_decoder = np.dot(W_q_decoder, output_eos)
k_eos_decoder = np.dot(W_k_decoder, output_eos)
v_eos_decoder = np.dot(W_v_decoder, output_eos)
print(q_eos_decoder)
print(k_eos_decoder)
print(v_eos_decoder)


[1.182 1.046]
[ 1.216 -1.992]
[-2.834 -2.332]


In [10]:
softmax_q_eos_decoder = softmax(np.array([np.dot(q_eos_decoder, k_eos_decoder.T)]))
print(softmax_q_eos_decoder)
output_decoder_1_eos = softmax_q_eos_decoder * v_eos_decoder
print(output_decoder_1_eos)

resi_decoder_1_eos = output_decoder_1_eos + output_eos
print(resi_decoder_1_eos)

[1.]
[-2.834 -2.332]
[-0.134 -2.672]


In [11]:
W_q_decoder_2 = np.array([[ 1.5,  0.3],
                          [-0.3,  1.0]])
W_k_encoder_2 = np.array([[-1.1, -0.3],
                          [ 0.3, -0.8]])
W_v_encoder_2 = np.array([[ 1.1, -1.2],
                          [ 0.6, -0.5]])

q_eos_decoder_2 = np.dot(W_q_decoder_2, resi_decoder_1_eos)
print(q_eos_decoder_2)


k_1_encoder_2 = np.dot(W_k_encoder_2, resi_encoder_1)
k_2_encoder_2 = np.dot(W_k_encoder_2, resi_encoder_2)
print(k_1_encoder_2)
print(k_2_encoder_2)


softmax_q_eos_decoder_2 = softmax(np.array([np.dot(q_eos_decoder_2, k_1_encoder_2.T), np.dot(q_eos_decoder_2, k_2_encoder_2.T)]))
print(softmax_q_eos_decoder_2)


v_1_encoder_2 = np.dot(W_v_encoder_2, resi_encoder_1)
v_2_encoder_2 = np.dot(W_v_encoder_2, resi_encoder_2)
print(v_1_encoder_2)
print(v_2_encoder_2)

output_decoder_2_eos = np.dot(softmax_q_eos_decoder_2, np.array([v_1_encoder_2, v_2_encoder_2]))
print(output_decoder_2_eos)

resi_decoder_2_eos = output_decoder_2_eos + resi_decoder_1_eos
print(resi_decoder_2_eos)

word_array_output = np.array([[-0.6, -2.0],
                              [ 0.8, -0.9],
                              [-0.1, -1.1],
                              [-1.0,  1.6]])
bias_output = np.array([-0.6, 1.4, -2.5, 0.5]).T
first_output = softmax(np.dot(word_array_output, resi_decoder_2_eos) + bias_output)
print(first_output)

[-1.0026 -2.6318]
[-4.48339797  2.10279804]
[-0.4524  1.3738]
[0.89310924 0.10689076]
[5.98039557 3.10779783]
[2.5794 1.1878]
[5.61686057 2.9025678 ]
[5.48286057 0.2305678 ]
[4.86988785e-05 9.99774851e-01 1.39017150e-04 3.74331197e-05]
